# [Get Rich or Buy Trying - Part 3](https://xpuzzlecorner.substack.com/p/get-rich-or-buy-trying-part-3)
## October 20, 2024

## Problem

_Consider the following stock behavior:_

_The stock’s value will change by a factor of (1+δ) from it’s pervious value each hour where δ is uniformly distributed from [-1/100, 1/100]. For example, after two hours, the value would be p_0 * δ_1 * δ_2. Note: these values are slightly different from Part 2._

_The question for this week is, what’s the average difference in performance between an oracle that can predict whether the stock will go up each hour vs the oracle that can predict whether the stock will go up each day? The oracles are to perform over 5 days, 7 hours each day._

## Solution

The situation is quite similar to [Part 2](https://github.com/xavierdurawa/XPC/blob/master/10-13-24/10-13-24.ipynb) except that the number of compounding periods is itself now a random number binomially distributed with $ p = .5 $ and $ N = 35 $ for the hourly case and $ N = 5 $ for the daily case. Recall from [Part 1](https://github.com/xavierdurawa/XPC/blob/master/10-06-24/10-06-24.ipynb) that we ended up performing a transformations over the binomial distribution. We can use a similar technique here since our number of compoundings are going to be binomially distributed. Let's let $ X $ be a uniformly distributed random variable on $ [0, 1/100] $, $ p_0=1 $, and $ Y_h $ be a binomially distributed random variable with $ p = .5 $ and $ N = 35 $. Then distribution of the stock price after 35 hours is $ (1 + X)^Y_h $. We can find the expected value of this distribution by finding the expected value of $ (1 + X)^{Y_h} $.

$$ E[(1 + X)^{Y_h}] = \displaystyle \sum_{y_h} \int (1 + x)^{y_h} f_X(x) f_{Y_h}(y_h) dx = \displaystyle \sum_{y_h=0}^{35} \int_0^{1/100} (1 + x)^{y_h} \cdot 100 \cdot \binom{35}{y_h} \cdot \frac{1}{2}^{35} dx = \displaystyle \sum_{y_h=0}^{35} \binom{35}{y_h} \cdot \frac{100}{2^{35}} \int_0^{1/100} (1 + x)^{y_h} dx $$ 

$$ = \displaystyle \sum_{y_h=0}^{35} \binom{35}{y_h} \cdot \frac{100}{2^{35}} \left[ \frac{(1 + x)^{y_h+1}}{y_h+1} \right]_0^{1/100} = \displaystyle \sum_{y_h=0}^{35} \binom{35}{y_h} \cdot \frac{100}{2^{35}} \left[ \frac{(1 + 1/100)^{y_h+1} - 1}{y_h+1} \right] = \boxed{ \displaystyle \sum_{y_h=0}^{35} \binom{35}{y_h} \cdot \frac{100}{2^{35}} \left[ \frac{(101/100)^{y_h+1} - 1}{y_h+1} \right] } $$

If we're not careful, we might assume we can obtain a very similar value for the daily case except that  $ Y_d $ is a binomially distributed random variable with $ p = .5 $ and $ N = 5 $. But this isn't quite right, since this would assume that the stock price is compounded daily. Instead, the stock price is compounded every hour. So instead we need to find the expected value of $ X^{Y_d} $ where $ Y_d $ is a binomially distributed random variable with $ p = .5 $ and $ N = 5 $ and $ X $ is distributed as $ U \sim [.99,1.01] $:

$$ \displaystyle f_{X}(x) = \frac{1}{(t-1)!} \left( \frac{1}{b-a} \right)^{t} \sum_{k=0}^{t} \binom{t}{k} (-1)^k c_k^{t-1} \cdot \mathbb{1}_{c_k > 0} $$

where $ t $ is the number of times the uniform variable is compounded ($ t=7 $ in this case), $ c_k = k \ln(b) + (t-k) \ln(a) - \ln(x) $, $ b=1.01 $ , and $ a=.99 $ (taken from [Part 2](https://github.com/xavierdurawa/XPC/blob/master/10-13-24/10-13-24.ipynb)). But with the way we set up $ Y_d $ we are only concerned with the part of the distribution where $ x > 1 $, so we need to rescale the distribution to account for this:


$$ E[(1 + X)^{Y_d}] = \displaystyle \sum_{y_d} \int x^{y_d} f_{X|X>1}(x) f_{Y_d}(y_d) dx = \displaystyle \sum_{y_d=0}^{5} \binom{5}{y_d} \frac{100}{2^{5}} \int_a^{b} x^{y_d} \cdot \frac{f_{X}(x)}{\int_1^b f_X(x)} dx $$

$ \int_1^b f_X(x) $ is just a scaling factor so let's come back to that later. Let's pull out all the constants to clean up the expression:

$$ \alpha = \frac{100^{7}}{2^{5}6! \int_1^b f_X(x)} $$

$$ E[(1 + X)^{Y_d}] = \alpha \displaystyle \sum_{y_d=0}^{5} \binom{5}{y_d} \int_1^{b} x^{y_d} \cdot \sum_{k=0}^{7} \binom{7}{k} (-1)^k c_k^{6} \cdot \mathbb{1}_{c_k > 0} dx $$

$$ E[(1 + X)^{Y_d}] = \alpha \displaystyle \sum_{y_d=0}^{5} \binom{5}{y_d} \sum_{k=0}^{7} \binom{7}{k} (-1)^k \int_1^{b} x^{y_d} \cdot c_k^{6} \cdot \mathbb{1}_{c_k > 0} dx $$

If we let $ \beta(k,t)= k \ln(b) + (t-k) \ln(a) $ We will ultimately have terms that we need to integrate of the form 
$$ \int_1^{b} x^{y_d} \cdot (\beta(k,t) - \ln(x))^6 \cdot \mathbb{1}_{c_k > 0} dx $$

$$ \int_1^{b} x^{y_d} \cdot (\beta(k,t) - \ln(x))^6 \cdot \mathbb{1}_{c_k > 0} dx $$

Sparing the details of the integration, which can be performed by iteratively doing integration by parts, we get:

$$
\begin{align*} I &= \sum_{k=0}^{6} \frac{6!}{(6 - k)! \, k!} \frac{1}{(y_d + 1)^{k + 1}} \left[ \xi^{y_d + 1} \left( \beta - \ln \xi \right)^{6 - k} - \beta^{6 - k} \right] \\ &\quad + \frac{6!}{(y_d + 1)^7} \left[ \frac{\xi^{y_d + 1} - 1}{y_d + 1} \right] 
\end{align*}
$$

Where $ \xi = \min(b,e^{\beta(k,t)}) $

Pulling it all together, we have:

$$ E[(1 + X)^{Y_d}] = \alpha \displaystyle \sum_{y_d=0}^{5} \binom{5}{y_d} \sum_{k=0}^{7} \binom{7}{k} (-1)^k I(k, y_d) $$

where $ I(k, y_d) $ is the integral above.


In [11]:
import math

def compute_sum():
    total = 0
    for y_h in range(36):  # y_h goes from 0 to 35 inclusive
        binomial_coefficient = math.comb(35, y_h)
        term = binomial_coefficient * (100 / (2 ** 35)) * ((101 / 100) ** (y_h + 1) - 1) / (y_h + 1)
        total += term
    return total

result = compute_sum()
print("The expected value E[(1 + X)^{Y_h}] is:", result)


The expected value E[(1 + X)^{Y_h}] is: 1.0926695823523307


In [8]:
# Simulate the hourly case

import random

def simulate_hourly():
    start = 1
    for _ in range(36):
        if random.random() < 0.5:
            continue
        delta = 1 + random.random() * (1/100)
        start *= delta
    return start

num_simulations = 10**6
total = sum(simulate_hourly() for _ in range(num_simulations))
print("The simulated expected value E[(1 + X)^{Y_h}] is:", total / num_simulations)

The simulated expected value E[(1 + X)^{Y_h}] is: 1.0940470458511669


In [13]:
import sympy as sp
import math
from scipy.special import binom

def compute_integral_symbolic(b, y_d, beta_k):
    # Define symbolic variables
    x = sp.symbols('x')
    integral_sum = 0
    factorial_6 = math.factorial(6)
    
    # Compute the symbolic integral result using iterated integration by parts
    for k in range(7):
        binom_coeff = binom(6, k)  # Binomial coefficient
        term = (
            binom_coeff * (b ** (y_d + 1) * (beta_k - sp.log(b)) ** (6 - k) / (y_d + 1) ** (7 - k))
            - binom_coeff * (beta_k ** (6 - k) / (y_d + 1) ** (7 - k))
        )
        integral_sum += term
    
    # Add the final constant term
    integral_sum += factorial_6 * (b ** (y_d + 1) - 1) / (y_d + 1) ** 7
    
    # Simplify and return the result
    return sp.simplify(integral_sum)

def compute_integral_numeric(b, y_d, beta_k):
    # Initialize the result
    integral_sum = 0
    factorial_6 = math.factorial(6)
    
    # Loop to compute each term numerically
    for k in range(7):
        binom_coeff = binom(6, k)  # Binomial coefficient
        term = (
            binom_coeff * (b ** (y_d + 1) * (beta_k - math.log(b)) ** (6 - k) / (y_d + 1) ** (7 - k))
            - binom_coeff * (beta_k ** (6 - k) / (y_d + 1) ** (7 - k))
        )
        integral_sum += term
    
    # Add the final constant term
    integral_sum += factorial_6 * (b ** (y_d + 1) - 1) / (y_d + 1) ** 7
    
    # Return the numeric result
    return integral_sum


In [15]:
# Example parameters
b_val = 2
y_d_val = 3
beta_k_val = 1.5

# Compute using symbolic approach
result_symbolic = compute_integral_symbolic(b_val, y_d_val, beta_k_val).evalf()
print("Symbolic Evaluation:", result_symbolic)

# Compute using numeric approach
result_numeric = compute_integral_numeric(b_val, y_d_val, beta_k_val)
print("Numeric Evaluation:", result_numeric)

# Compare results
print("Difference:", abs(result_symbolic - result_numeric))


Symbolic Evaluation: 11.0163013735153
Numeric Evaluation: 11.0163013735153
Difference: 1.77635683940025e-15


In [20]:
def simulate_daily():
    start = 1
    for _ in range(5):
        daily = 1
        for _ in range(7):
            delta = (random.random() - 0.5) / 50
            daily *= 1 + delta
        if daily > 1:
            start *= daily
    return start

num_simulations = 10**6
total = sum(simulate_daily() for _ in range(num_simulations))
print("The simulated expected value E[(1 + X)^{Y_d}] is:", total / num_simulations)

The simulated expected value E[(1 + X)^{Y_d}] is: 1.031070097597957


In [ ]:
#TODO: Needs to be fixed

def compute_expectation(b_value=1.01):
    # Constants
    alpha = (100 ** 7) / (2 ** 5 * math.factorial(6))

    # Compute E[(1 + X)^{Y_d}]
    expectation = 0
    for y_d in range(6):  # y_d goes from 0 to 5
        for k in range(8):  # k goes from 0 to 7
            binom_5_y_d = binom(5, y_d)
            binom_7_k = binom(7, k)
            I_k_y_d = compute_integral_numeric(b_value, y_d, k)
            term = binom_5_y_d * binom_7_k * ((-1) ** k) * I_k_y_d
            expectation += term

    # Multiply by alpha
    expectation *= alpha
    return expectation

# Example usage
result = compute_expectation(b_value=1.01)  # You can adjust b_value as needed
print("E[(1 + X)^{Y_d}] =", result)

E[(1 + X)^{Y_d}] = -0.10272743304155321


## Extra Credit

_What about the actual distribution of the stock price after a week?_